<a href="https://colab.research.google.com/github/Jess-pm/machine-learning/blob/main/Brain_tumor_detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

# Import Important libraries

import os
import cv2
import numpy as np
import tensorflow as tf
import gradio as gr
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [3]:
# Define the paths to your dataset folders
dataset_path = '/content/dataset'
yes_folder = os.path.join(dataset_path, 'yes')
no_folder = os.path.join(dataset_path, 'no')

# Initialize lists to store images and labels
tr_images = []
tr_labels = []
ts_images = []
ts_labels = []

# Function to load images from a folder
def load_images_from_folder(folder, is_training=True):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize image to (224, 224) image size has an impact on the quality of the outcome as pixel details may be lost when resized too small. depending on the requirements.
        img = img / 255.0  # Normalize pixel values
        images.append(img)
        if is_training:
            if folder.endswith('yes'):
                labels.append(1)  # Yes folder (tumor)
            elif folder.endswith('no'):
                labels.append(0)  # No folder (no tumor)
    return images, labels

# Load training images and labels
tr_images_yes, tr_labels_yes = load_images_from_folder(yes_folder)
tr_images_no, tr_labels_no = load_images_from_folder(no_folder)
tr_images.extend(tr_images_yes + tr_images_no)
tr_labels.extend(tr_labels_yes + tr_labels_no)

# Load testing images and labels (you can modify this based on your split)
ts_images.extend(tr_images[-10:])  # Using last 10 images for testing. simple split for demo purpose. should use train_test_split code.
ts_labels.extend(tr_labels[-10:])

# Convert lists to numpy arrays
tr_images = np.array(tr_images)
tr_labels = np.array(tr_labels)
ts_images = np.array(ts_images)
ts_labels = np.array(ts_labels)

# Print shapes to verify
print("Training Images Shape:", tr_images.shape)
print("Training Labels Shape:", tr_labels.shape)
print("Testing Images Shape:", ts_images.shape)
print("Testing Labels Shape:", ts_labels.shape)

Training Images Shape: (253, 224, 224, 3)
Training Labels Shape: (253,)
Testing Images Shape: (10, 224, 224, 3)
Testing Labels Shape: (10,)


In [4]:
# Build the CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') #recap, sigmoid means >=0.5 means yes, <0.5 no.
    #softmax also can use here, and use 2 neurons instead. then it will be output class = yes and no (2).
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Compile the model

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
# Train the model
model.fit(tr_images, tr_labels, epochs=10, validation_data=(ts_images, ts_labels))

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 33s 4s/step - accuracy: 0.5482 - loss: 0.7437 - val_accuracy: 1.0000 - val_loss: 0.4259
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.7807 - loss: 0.4980 - val_accuracy: 0.7000 - val_loss: 0.4397
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.8446 - loss: 0.3913 - val_accuracy: 0.5000 - val_loss: 0.6173
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.8832 - loss: 0.2932 - val_accuracy: 0.4000 - val_loss: 0.7868
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.9062 - loss: 0.2403 - val_accuracy: 0.8000 - val_loss: 0.3560
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 35s 4s/step - accuracy: 0.9239 - loss: 0.1880 - val_accuracy: 1.0000 - val_loss: 0.0872
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.9563 - loss: 0.0875 - val_accuracy: 1.0000 - val_loss: 0.0152
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.9986 - loss: 0.0314 - val_accuracy: 1.0000 - val_loss: 0.0155
Epoch 9/

In [7]:
# Evaluate the model
test_loss, test_acc = model.evaluate(ts_images, ts_labels)

print('Test accuracy:', test_acc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 1.0000 - loss: 0.0708
Test accuracy: 1.0


In [9]:
# Define a function for Gradio
def predict_image(img):
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.image.resize(img, (224, 224))  # Resize to match model's expected sizing
    img = img / 255.0  # Normalize pixel values
    prediction = model.predict(img)
    return "Brain Tumor" if prediction[0][0] > 0.5 else "No Brain Tumor"

# Create Gradio interface
iface = gr.Interface(fn=predict_image, inputs="image", outputs="text")
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba90e93ea4366f109a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
